 ## QUESTION2

In [7]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.preprocessing import MinMaxScaler #normalizing data
from sklearn.metrics import classification_report

In [8]:
df = pd.read_csv("C:\\Users\\Hp\\Downloads\\q2_Fraud_check.csv")

df=pd.get_dummies(df,columns=['Undergrad', 'Marital.Status', 'Urban'], drop_first=True) # Categorical Variable Handling

def target_transform(tax): #Target Variable Transformation
    if tax<=30000:
        return 0
    else:
        return 1
df['Taxable.Income'] = df['Taxable.Income'].apply(target_transform)

scaler = MinMaxScaler() #Feature Scaling
df[['Work.Experience', 'City.Population']] = scaler.fit_transform(df[['Work.Experience', 'City.Population']])

feature_cols = list(df.columns)  #dataspiltting
feature_cols.remove('Taxable.Income')
X=df[feature_cols]
y= df['Taxable.Income']

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.19      0.28      0.22        36
           1       0.80      0.70      0.75       144

    accuracy                           0.62       180
   macro avg       0.49      0.49      0.49       180
weighted avg       0.67      0.62      0.64       180



# Question1

In [9]:
import pandas as pd
import re                                       #DATA PRE-PROCESSING
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np
df1 = pd.read_csv("C:\\Users\\Hp\\Downloads\\Q1_property.csv", delimiter=';', encoding='utf-8')
desired_city = 'Islamabad'
filtered_data = df1[df1['city'] == desired_city]

filtered_data['date_added'] = pd.to_datetime(filtered_data['date_added'])

marla_to_sqft = 272  
kanal_to_sqft = 5445 
# Function to convert area values
def convert_area(row):
    value = row['area']
    numeric_value = re.findall(r'\d*\.?\d+', value)  # Extract numeric value including decimals
    numeric_value = float(numeric_value[0])  # Convert the extracted numeric value to float
    if 'Marla' in value:
        return numeric_value * marla_to_sqft  # Calculate area in square feet (as float)
    elif 'Kanal' in value:
        return numeric_value * kanal_to_sqft  # Calculate area in square feet (as float)
    else:
        return None

filtered_data['area'] = filtered_data.apply(convert_area, axis=1)
filtered_data['area'] = pd.to_numeric(filtered_data['area'], errors='coerce')

mode = filtered_data['agency'].mode()[0]  
filtered_data['agency'].fillna(mode, inplace=True)
mode = filtered_data['agent'].mode()[0]  
filtered_data['agent'].fillna(mode, inplace=True)

median = filtered_data['area'].median()
filtered_data['area'].replace(0, median, inplace=True)
median = filtered_data['baths'].median()
filtered_data['baths'].replace(0, median, inplace=True)
median = filtered_data['bedrooms'].median()
filtered_data['bedrooms'].replace(0, median, inplace=True)

numerical_columns = filtered_data.select_dtypes(include=['int64', 'float64'])
# Calculate the IQR for all numerical columns
Q1 = numerical_columns.quantile(0.25)
Q3 = numerical_columns.quantile(0.75)
IQR = Q3 - Q1
# Define the bounds for outliers using the IQR method
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
# Identify outliers in any column using any outlier criterion
outliers = ((numerical_columns < lower_bound) | (numerical_columns > upper_bound)).any(axis=1)
# Get rows containing outliers
outliers_data = filtered_data[outliers]
no_outliers = filtered_data[~outliers]

#print(filtered_data.eq(0).sum())
#print(filtered_data.isnull().sum())
#print(filtered_data.dtypes)
#print(filtered_data)

C:\Users\Hp\AppData\Local\Temp\ipykernel_4664\1386146588.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['date_added'] = pd.to_datetime(filtered_data['date_added'])
C:\Users\Hp\AppData\Local\Temp\ipykernel_4664\1386146588.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['area'] = filtered_data.apply(convert_area, axis=1)
C:\Users\Hp\AppData\Local\Temp\ipykernel_4664\1386146588.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [10]:
correlation = filtered_data.corr().abs().stack()                             #EDA
max_corr_pair = correlation.idxmax()  # Retrieve the index of the maximum correlation
# Print the pair of variables with the highest correlation
print(f"The pair of variables with the highest correlation are: {max_corr_pair} with a correlation of {correlation[max_corr_pair]:.2f}")
min_corr_pair = correlation.idxmin()
# Print the pair of variables with the lowest correlation
print(f"The pair of variables with the lowest correlation are: {min_corr_pair} with a correlation of {correlation[min_corr_pair]:.2f}")

# Group data by agent and calculate the count of properties listed and the average price
agent_property_count = filtered_data.groupby('agent')['property_id'].count()
agent_average_price = filtered_data.groupby('agent')['price'].mean()
# Calculate the correlation between property count and average price
correlation = agent_property_count.corr(agent_average_price)
print(f"The correlation between the number of properties listed and average property price is: {correlation:.2f}")

The pair of variables with the highest correlation are: ('property_id', 'property_id') with a correlation of 1.00
The pair of variables with the lowest correlation are: ('price', 'longitude') with a correlation of 0.00
The correlation between the number of properties listed and average property price is: -0.01


C:\Users\Hp\AppData\Local\Temp\ipykernel_4664\1479380111.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = filtered_data.corr().abs().stack()                             #EDA


In [11]:
filtered_data['price_per_sq_meter'] = filtered_data['price'] / filtered_data['area']
#print(filtered_data['price_per_sq_meter'])

filtered_data['month_added'] = filtered_data['date_added'].dt.month     #FEATURE ENGINEERING
filtered_data['quarter_added'] = filtered_data['date_added'].dt.quarter

numerical_columns = filtered_data.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
filtered_data[numerical_columns] = scaler.fit_transform(filtered_data[numerical_columns])

encoded = pd.get_dummies(filtered_data['agency'], prefix='agency')
filtered_data = pd.concat([filtered_data, encoded], axis=1)
encoded = pd.get_dummies(filtered_data['agent'], prefix='agent')
filtered_data = pd.concat([filtered_data, encoded], axis=1)

C:\Users\Hp\AppData\Local\Temp\ipykernel_4664\114608405.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['price_per_sq_meter'] = filtered_data['price'] / filtered_data['area']
C:\Users\Hp\AppData\Local\Temp\ipykernel_4664\114608405.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['month_added'] = filtered_data['date_added'].dt.month     #FEATURE ENGINEERING
C:\Users\Hp\AppData\Local\Temp\ipykernel_4664\114608405.py:5: SettingWithCopyWarning: 
A value is trying to be set on 

In [12]:
feature = ['area', 'bedrooms', 'baths']
X = filtered_data[feature]                #MODEL TRAINING AND EVALUTION

y = filtered_data['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
lm = LinearRegression()
model = lm.fit(X_train, y_train)
print(f'alpha = {model.intercept_}')
print(f'betas = {model.coef_}')
predictions = model.predict(X_test)
print("predictions:",predictions)
# Calculate evaluation metrics
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, predictions)

print(f'Mean Squared Error on test set: {mse}')
print(f'Mean Absolute Error on test set: {mae}')
print(f'Root Mean Squared Error on test set: {rmse}')
print(f'Mean Absolute Percentage Error on test set: {mape}')

alpha = 0.0034604399645817544
betas = [0.06372039 0.29072746 0.0502246 ]
predictions: [-0.12486499 -0.12108013  0.28581014 ... -0.1286414  -0.51320228
 -0.12947036]
Mean Squared Error on test set: 0.8070934904112435
Mean Absolute Error on test set: 0.4126269776875249
Root Mean Squared Error on test set: 0.8983838213209561
Mean Absolute Percentage Error on test set: 2.9450040832439126
